In [1]:
import configparser
import json
import spotipy
import spotipy.util as util
import pandas as pd
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
import bamboolib

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open('../data/raw/spotify_creds.json') as f:
    spotify_creds = json.load(f)

In [4]:
client_id = spotify_creds['client_id']
client_secret = spotify_creds['client_secret']
username = spotify_creds['username']
#scope = spotify_creds['scope']
#redirect_uri = spotify_creds['redirect_url']

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

In [5]:
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [6]:
def connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri):
    
    client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

    if token:
        sp = spotipy.Spotify(auth=token)
    else:
        print("Can't get token for", username)
        
    return sp

In [7]:
sp = connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri)

In [8]:
def get_saved_library():
    
    df_saved_tracks = pd.DataFrame()
    track_list = ''
    added_ts_list = []
    artist_list = []
    title_list = []
    popularity_list = []
    album_cover_list = []
    more_songs = True
    offset_index = 0

    while more_songs:
        songs = sp.current_user_saved_tracks(offset=offset_index)
        for song in songs['items']:
            #join track ids to a string for audio_features function
            track_list += song['track']['id'] +','
            #get the time when the song was added
            added_ts_list.append(song['added_at'])
            #get the title of the song
            title_list.append(song['track']['name'])
            #get popularity
            popularity_list.append(song['track']['popularity'])
            # get album cover
            album_cover_url = song['track']['album']['images'][0]['url']
            #response = requests.get(album_cover_url)
            #img = Image.open(BytesIO(response.content))
            album_cover_list.append(album_cover_url)
            #get all the artists in the song
            artists = song['track']['artists']
            artists_name = ''
            for artist in artists:
                artists_name += artist['name']  + ','
            artist_list.append(artists_name[:-1])
        track_list = ''
        if songs['next'] == None:
            # no more songs in playlist
            more_songs = False
        else:
            # get the next n songs
            offset_index += songs['limit']
    #include timestamp added, title and artists of a song
    df_saved_tracks['song_title'] = title_list
    df_saved_tracks['artists'] = artist_list
    df_saved_tracks['date_added'] = added_ts_list
    df_saved_tracks['album_cover'] = album_cover_list
    df_saved_tracks['popularity'] = popularity_list
    
    return df_saved_tracks

In [9]:
df = get_saved_library()

In [5]:
scope = 'user-read-recently-played'
redirect_uri = 'http://localhost:8886/callback'
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

In [6]:
sp = spotipy.Spotify(auth=token)

In [7]:
songs = sp.current_user_recently_played(limit=50)

In [8]:
def get_recently_played():
    
    df_saved_tracks = pd.DataFrame()
    track_list = ''
    played_ts_list = []
    artist_list = []
    title_list = []
    popularity_list = []
    album_cover_list = []
    more_songs = True
    offset_index = 0

    while more_songs:
        songs = sp.current_user_recently_played(limit=50)
        for song in songs['items']:
            #join track ids to a string for audio_features function
            track_list += song['track']['id'] +','
            #get the time when the song was added
            played_ts_list.append(song['played_at'])
            #get the title of the song
            title_list.append(song['track']['name'])
            #get popularity
            popularity_list.append(song['track']['popularity'])
            # get album cover
            album_cover_url = song['track']['album']['images'][0]['url']
            #response = requests.get(album_cover_url)
            #img = Image.open(BytesIO(response.content))
            album_cover_list.append(album_cover_url)
            #get all the artists in the song
            artists = song['track']['artists']
            artists_name = ''
            for artist in artists:
                artists_name += artist['name']  + ','
            artist_list.append(artists_name[:-1])
        track_list = ''
        if songs['next'] == None:
            # no more songs in playlist
            more_songs = False
        else:
            # get the next n songs
            offset_index += songs['limit']
    #include timestamp added, title and artists of a song
    df_saved_tracks['song_title'] = title_list
    df_saved_tracks['artists'] = artist_list
    df_saved_tracks['played_at'] = played_ts_list
    df_saved_tracks['album_cover'] = album_cover_list
    df_saved_tracks['popularity'] = popularity_list
    
    return df_saved_tracks

In [13]:
def get_recently_played():

    df_saved_tracks = pd.DataFrame()
    track_list = ''
    played_ts_list = []
    artist_list = []
    title_list = []
    popularity_list = []
    album_cover_list = []
    more_songs = True


    songs = sp.current_user_recently_played(limit=50)
    for song in songs['items']:
                #join track ids to a string for audio_features function
        track_list += song['track']['id'] +','
                #get the time when the song was added
        played_ts_list.append(song['played_at'])
                #get the title of the song
        title_list.append(song['track']['name'])
                #get popularity
        popularity_list.append(song['track']['popularity'])
                # get album cover
        album_cover_url = song['track']['album']['images'][0]['url']
                #response = requests.get(album_cover_url)
                #img = Image.open(BytesIO(response.content))
        album_cover_list.append(album_cover_url)
                #get all the artists in the song
        artists = song['track']['artists']
        artists_name = ''
        for artist in artists:
            artists_name += artist['name']  + ','
        artist_list.append(artists_name[:-1])
        track_list = ''
        #include timestamp added, title and artists of a song
    df_saved_tracks['song_title'] = title_list
    df_saved_tracks['artists'] = artist_list
    df_saved_tracks['played_at'] = played_ts_list
    df_saved_tracks['album_cover'] = album_cover_list
    df_saved_tracks['popularity'] = popularity_list
    
    return df_saved_tracks

In [28]:
df = get_recently_played()

In [29]:
df

,song_title,artists,played_at,album_cover,popularity
0,If I Ain't Got You,Alicia Keys,2022-02-28T00:04:40.910Z,https://i.scdn.co/image/ab67616d0000b27356ff19...,83
1,If I Ain't Got You - Acoustic,John Adams,2022-02-28T00:00:56.600Z,https://i.scdn.co/image/ab67616d0000b273d2f483...,54
2,If I Ain't Got You - Acoustic,John Adams,2022-02-27T23:58:07.005Z,https://i.scdn.co/image/ab67616d0000b273d2f483...,54
3,Walk Me Home,"Said The Sky,ILLENIUM,Chelsea Cutler",2022-02-27T17:45:09.993Z,https://i.scdn.co/image/ab67616d0000b2731a15fe...,72
4,Okay,"Nicky Romero,MARF,Wulf",2022-02-27T03:36:55.993Z,https://i.scdn.co/image/ab67616d0000b273dd4a02...,65
5,Blame Myself,"ILLENIUM,Tori Kelly",2022-02-27T03:35:21.422Z,https://i.scdn.co/image/ab67616d0000b273700ed2...,63
6,Blame Myself,"ILLENIUM,Tori Kelly",2022-02-27T03:30:53.981Z,https://i.scdn.co/image/ab67616d0000b273700ed2...,63
7,Lasting Lover - Tiësto Remix,"Sigala,James Arthur,Tiësto",2022-02-27T03:27:04.027Z,https://i.scdn.co/image/ab67616d0000b2738f57d5...,61
8,痴心絕對,Sam Lee,2022-02-27T03:19:24.782Z,https://i.scdn.co/image/ab67616d0000b273e34baf...,54
9,Close Your Eyes - VIP Mix,"KSHMR,Tungevaag",2022-02-26T19:54:46.450Z,https://i.scdn.co/image/ab67616d0000b273d6c479...,56
